In [1]:
from keras.models import load_model
from keras.models import Sequential
from src.average_precision import convert_to_names
from keras.preprocessing import sequence
from datetime import datetime
import pandas as pd
import numpy as np
import pickle

Using TensorFlow backend.


In [2]:
MODEL_NAME = "lstm_2016-12-03 11:50:11_FULL"

In [3]:
X = pickle.load( open( "data/training_full.pickle", "rb" ) )
ids = pickle.load( open( "data/training_ids.pickle", "rb" ) )

In [4]:
test_ids = pd.read_csv("data/test_ver2.csv", usecols=[1])

In [5]:
X =sequence.pad_sequences(X, maxlen=5, dtype='float')

In [6]:
best_model_path = "models/{0}".format(MODEL_NAME)
model = load_model(best_model_path)

In [8]:
preds = np.round(model.predict_proba(X, batch_size=512), 2)

937984/938418 [============================>.] - ETA: 0s

In [9]:
predicted_names = []
for i, p in enumerate(preds):
    predicted_names.append(convert_to_names(p, 0.01))

In [10]:
submission =  pd.DataFrame()

In [11]:
submission["ncodpers"] = ids

In [12]:
submission["added_products"] = predicted_names

In [14]:
def flatten_list(arr):
    return " ".join(arr)

In [15]:
submission["added_products"] = submission["added_products"].apply(flatten_list)

In [16]:
final_submission = test_ids.merge(submission, how="left")

In [83]:
final_submission.to_csv("submissions/submission_{0}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")), index=False)

In [93]:
final_submission[final_submission["ncodpers"] == 660217].values

array([[660217,
        'ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_ecue_fin_ult1 ind_tjcr_fin_ult1 ind_nomina_ult1 ind_nom_pens_ult1 ind_recibo_ult1']], dtype=object)

In [98]:
y_cols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
          'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
          'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
          'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
          'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
          'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
          'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
          'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [99]:
preds_df = pd.DataFrame(preds)

In [101]:
preds_df.columns = y_cols

In [17]:
len(final_submission)

929615